In [16]:
import requests
%matplotlib inline
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json
import urllib
import numpy as np
import string
import pyodbc

In [18]:
cnxn = pyodbc.connect(DSN="IIW",ansi=True)
CM = pd.io.sql.read_sql_query(""" 

with temp1 as (
SELECT
	CASE 
        WHEN regexp_match_count(substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)),'&') = 0
        THEN substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)+9)
        WHEN regexp_match_count(substr(lower(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)),'&') > 0
        THEN substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)+9, instr(upper(FIRST_PAGE_COMPL_URL),'&',instr(upper(FIRST_PAGE_COMPL_URL),'CM_MMCA1=',1,1),1) - instr(upper(FIRST_PAGE_COMPL_URL),'CM_MMCA1=',1,1)-9)
        ELSE 'no-code' 
    	END AS PROJ_CD
	,FIRST_PAGE_URL
	,URN_MPW_VISIT
	
	FROM BACC_PRD_IDM_IIW.V2DIGT2.V_FCT_VISIT_PQ1
	WHERE lower(FIRST_PAGE_COMPL_URL) LIKE '%cm_mmca1%'
	AND VISIT_CAT_CD = 0
	AND PROJ_CD != ''

	GROUP BY 
		PROJ_CD
		,FIRST_PAGE_URL
		,URN_MPW_VISIT
		
union all

SELECT
	CASE 
        WHEN regexp_match_count(substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)),'&') = 0
        THEN substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)+9)
        WHEN regexp_match_count(substr(lower(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)),'&') > 0
        THEN substr(upper(FIRST_PAGE_COMPL_URL),instr(upper(FIRST_PAGE_COMPL_URL), 'CM_MMCA1=',1,1)+9, instr(upper(FIRST_PAGE_COMPL_URL),'&',instr(upper(FIRST_PAGE_COMPL_URL),'CM_MMCA1=',1,1),1) - instr(upper(FIRST_PAGE_COMPL_URL),'CM_MMCA1=',1,1)-9)
        ELSE 'no-code' 
    	END AS PROJ_CD
	,FIRST_PAGE_URL
	,URN_MPW_VISIT
	
	FROM BACC_PRD_IDM_IIW.V2DIGT2.V_FCT_VISIT_CQ
	WHERE lower(FIRST_PAGE_COMPL_URL) LIKE '%cm_mmca1%'
	AND VISIT_CAT_CD = 0
	AND PROJ_CD != ''

	GROUP BY 
		PROJ_CD
		,FIRST_PAGE_URL
		,URN_MPW_VISIT),

temp2 as (
SELECT
	temp1.PROJ_CD
	,temp1.FIRST_PAGE_URL
	,COUNT(pv.SESN_ID) AS VISITS
	,SUM(case when pv.PAGE_ENGAGEMT_TYPE >= 1 then 1 else 0 end) AS ENG
	
	FROM temp1
	JOIN BACC_PRD_IDM_IIW.V2DIGT2.V_FCT_PAGE_VW_PQ1 pv
		ON pv.URN_MPW_VISIT = temp1.URN_MPW_VISIT
	
	WHERE pv.PAGE_VW_COREMETRICS_TS BETWEEN '2018-10-01 00:00:00' AND '2018-12-31 23:59:59'
	AND pv.FIRST_PAGE_FLG_BNRY = 1 
		
	GROUP BY
		temp1.PROJ_CD
		,temp1.FIRST_PAGE_URL

union all

SELECT
	temp1.PROJ_CD
	,temp1.FIRST_PAGE_URL
	,COUNT(pv.SESN_ID) AS VISITS
	,SUM(case when pv.PAGE_ENGAGEMT_TYPE >= 1 then 1 else 0 end) AS ENG
	
	FROM temp1
	JOIN BACC_PRD_IDM_IIW.V2DIGT2.V_FCT_PAGE_VW_CQ pv
		ON pv.URN_MPW_VISIT = temp1.URN_MPW_VISIT
	
	WHERE pv.PAGE_VW_COREMETRICS_TS BETWEEN '2019-01-01 00:00:00' AND '2019-02-02 23:59:59'
	AND pv.FIRST_PAGE_FLG_BNRY = 1 
		
	GROUP BY
		temp1.PROJ_CD
		,temp1.FIRST_PAGE_URL)

SELECT 
	d.PLAN_CD
	,d.PLAN_NAME
	,d.PLAN_DSCR
	,d.PROJ_NAME
	,d.PROJ_DSCR
	,d.PROJ_STAT_CD
	,d.PROJ_CD
	,temp2.FIRST_PAGE_URL
	,temp2.VISITS
	,temp2.ENG

    FROM temp2
		JOIN BACC_PRD_IDM_IIW.V2INTR2.V_DIM_PROG d
		ON d.PROJ_CD = temp2.PROJ_CD
 		
		
        WHERE d.PROJ_STAT_CD = 'IN_PROGRESS'
        
        GROUP BY 
            d.PLAN_CD
			,d.PLAN_NAME
			,d.PLAN_DSCR
			,d.PROJ_NAME
			,d.PROJ_DSCR
			,d.PROJ_STAT_CD
			,d.PROJ_CD
			,temp2.FIRST_PAGE_URL
			,temp2.VISITS
			,temp2.ENG
            
    ORDER BY VISITS DESC

""", con=cnxn)

cnxn.close()
CM.head(10)

,PLAN_CD,PLAN_NAME,PLAN_DSCR,PROJ_NAME,PROJ_DSCR,PROJ_STAT_CD,PROJ_CD,FIRST_PAGE_URL,VISITS,ENG
0,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,2182503,595038.0
1,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,734257,80358.0
2,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud/why-ibm,575951,31438.0
3,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027HR_CAdv_Watson,Develop Watson content that will run on paid m...,IN_PROGRESS,000027HR ...,www.ibm.com/watson/campaign,437351,33382.0
4,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,415288,199494.0
5,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,292341,55396.0
6,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/yourcloud.html,177051,16313.0
7,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,151141,51821.0
8,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,143606,12304.0
9,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/thought-leadership/jp-ja/your-cloud,139309,11579.0


In [19]:
len(CM)

23104

In [20]:
CM.to_csv('C:\\Users\\IBM_ADMIN\\Documents\\Alerts\\ALERT 2 - Unregistered URLs\\CM_20190202.csv', header=True, index=False, index_label=None)

In [21]:
from ibmdbpy import IdaDataBase, IdaDataFrame

cnxn = IdaDataBase(dsn="DB2DR")

DR = cnxn.ida_query("""

    SELECT DISTINCT URL_ID, PAGE_TYPE
    FROM "INT".URL_HEADER;

    """)

cnxn.close()
DR.head(10)

Connection closed.


,url_id,page_type
0,abstracts/sg246374.html,NMK
1,ac2016.business.weather.com,MKT
2,appconnect.ibmcloud.com/applications/box,NMK
3,appconnect.ibmcloud.com/applications/salesforc...,NMK
4,appconnect.ibmcloud.com/pricing,NMK
5,assistant-demo.ng.bluemix.net,MKT
6,biz.weather.com/2015maxreality2webinar_10a_tha...,MKT
7,biz.weather.com/2015maxtrafficwebinar_10a_than...,MKT
8,biz.weather.com/eb_q2_201704_av_na_earthday_th...,MKT
9,biz.weather.com/eb_q2_201704_av_na_hubcast_ale...,MKT


In [22]:
len(DR)

708654

In [23]:
DR2 = DR.drop_duplicates(subset='url_id', keep='last')
DR2.head(10)

,url_id,page_type
0,abstracts/sg246374.html,NMK
1,ac2016.business.weather.com,MKT
2,appconnect.ibmcloud.com/applications/box,NMK
3,appconnect.ibmcloud.com/applications/salesforc...,NMK
4,appconnect.ibmcloud.com/pricing,NMK
5,assistant-demo.ng.bluemix.net,MKT
6,biz.weather.com/2015maxreality2webinar_10a_tha...,MKT
7,biz.weather.com/2015maxtrafficwebinar_10a_than...,MKT
8,biz.weather.com/eb_q2_201704_av_na_earthday_th...,MKT
9,biz.weather.com/eb_q2_201704_av_na_hubcast_ale...,MKT


In [24]:
len(DR2)

708580

In [25]:
DR2.to_csv('C:\\Users\\IBM_ADMIN\\Documents\\Alerts\\ALERT 2 - Unregistered URLs\\DR_20190202.csv', header=True, index=False, index_label=None)

In [26]:
CM_DR = pd.merge(CM, DR2, left_on='FIRST_PAGE_URL', right_on='url_id', how='left')
CM_DRdf = CM_DR.rename(columns=({'url_id':'dr_url'}))
CM_DRdf.sort_values('dr_url')
CM_DRdf.head(10)

,PLAN_CD,PLAN_NAME,PLAN_DSCR,PROJ_NAME,PROJ_DSCR,PROJ_STAT_CD,PROJ_CD,FIRST_PAGE_URL,VISITS,ENG,dr_url,page_type
0,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,2182503,595038.0,www-31.ibm.com/ibm/cn/smart,MKT
1,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,734257,80358.0,www.ibm.com/thought-leadership/smart,MKT
2,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud/why-ibm,575951,31438.0,www.ibm.com/cloud/why-ibm,MKT
3,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027HR_CAdv_Watson,Develop Watson content that will run on paid m...,IN_PROGRESS,000027HR ...,www.ibm.com/watson/campaign,437351,33382.0,www.ibm.com/watson/campaign,MKT
4,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,415288,199494.0,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,MKT
5,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,292341,55396.0,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,MKT
6,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/yourcloud.html,177051,16313.0,www.ibm.com/cloud-computing/cn-zh/yourcloud.html,MKT
7,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,151141,51821.0,www-31.ibm.com/ibm/cn/smart,MKT
8,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,143606,12304.0,www.ibm.com/thought-leadership/smart,MKT
9,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/thought-leadership/jp-ja/your-cloud,139309,11579.0,www.ibm.com/thought-leadership/jp-ja/your-cloud,MKT


In [27]:
CM_DRdf["mapping"] = CM_DRdf["PROJ_CD"] + CM_DRdf["FIRST_PAGE_URL"]
CM_DRdf = CM_DRdf.drop_duplicates(subset='mapping', keep='last')
del CM_DRdf['mapping']
CM_DRdf.head(10)

,PLAN_CD,PLAN_NAME,PLAN_DSCR,PROJ_NAME,PROJ_DSCR,PROJ_STAT_CD,PROJ_CD,FIRST_PAGE_URL,VISITS,ENG,dr_url,page_type
7,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,151141,51821.0,www-31.ibm.com/ibm/cn/smart,MKT
8,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,143606,12304.0,www.ibm.com/thought-leadership/smart,MKT
12,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud/why-ibm,118771,6833.0,www.ibm.com/cloud/why-ibm,MKT
16,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027HR_CAdv_Watson,Develop Watson content that will run on paid m...,IN_PROGRESS,000027HR ...,www.ibm.com/watson/campaign,96653,6322.0,www.ibm.com/watson/campaign,MKT
32,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/thought-leadership/jp-ja/your-cloud,48762,3241.0,www.ibm.com/thought-leadership/jp-ja/your-cloud,MKT
34,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,46264,5563.0,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,MKT
41,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart/de-de,37505,2332.0,www.ibm.com/thought-leadership/smart/de-de,MKT
42,PLN1153,Hybrid Cloud,2019 Hybrid Cloud,000028KG_WW_Cloud Narrative Campaign,Cloud/Analytics Umbrella Project to support th...,IN_PROGRESS,000028KG ...,www.ibm.com/cloud/smartpapers/cloud-native-app...,33823,27316.0,www.ibm.com/cloud/smartpapers/cloud-native-app...,MKT
43,PLN1153,Hybrid Cloud,2019 Hybrid Cloud,000033JI CN Cloud Narrative Campaign,,IN_PROGRESS,000033JI ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,33070,13772.0,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,MKT
63,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,23758,4854.0,www.ibm.com/cloud-computing/cn-zh/smartpapers/...,MKT


In [28]:
len(CM_DRdf)

17231

In [29]:
CM_DRdf['alert_id'] = np.where(CM_DRdf['dr_url'].isnull(), 'DR_1', 'NA')
CM_DRdf2= CM_DRdf[CM_DRdf['PROJ_CD'].notnull()]
CM_DRdf2.head(5)

,PLAN_CD,PLAN_NAME,PLAN_DSCR,PROJ_NAME,PROJ_DSCR,PROJ_STAT_CD,PROJ_CD,FIRST_PAGE_URL,VISITS,ENG,dr_url,page_type,alert_id
7,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www-31.ibm.com/ibm/cn/smart,151141,51821.0,www-31.ibm.com/ibm/cn/smart,MKT,NA
8,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,000027JP ...,www.ibm.com/thought-leadership/smart,143606,12304.0,www.ibm.com/thought-leadership/smart,MKT,NA
12,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/cloud/why-ibm,118771,6833.0,www.ibm.com/cloud/why-ibm,MKT,NA
16,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027HR_CAdv_Watson,Develop Watson content that will run on paid m...,IN_PROGRESS,000027HR ...,www.ibm.com/watson/campaign,96653,6322.0,www.ibm.com/watson/campaign,MKT,NA
32,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JN_CAdv_Cloud,Develop Cloud content that will run on paid media,IN_PROGRESS,000027JN ...,www.ibm.com/thought-leadership/jp-ja/your-cloud,48762,3241.0,www.ibm.com/thought-leadership/jp-ja/your-cloud,MKT,NA


In [30]:
len(CM_DRdf2)

17231

In [31]:
CM_DRdf2.to_csv('C:\\Users\IBM_ADMIN\\Documents\\Alerts\\ALERT 2 - Unregistered URLs\\cm_dr_20190202.csv', header=True, index=False, index_label=None)

In [32]:
CM_DRdf2.columns

Index(['PLAN_CD', 'PLAN_NAME', 'PLAN_DSCR', 'PROJ_NAME', 'PROJ_DSCR',
       'PROJ_STAT_CD', 'PROJ_CD', 'FIRST_PAGE_URL', 'VISITS', 'ENG', 'dr_url',
       'page_type', 'alert_id'],
      dtype='object')

In [33]:
import pandasql as ps

CM_DRdf3 = """
    
    SELECT 
        PLAN_CD
        ,PLAN_NAME
        ,PLAN_DSCR
        ,PROJ_CD
        ,PROJ_NAME
        ,PROJ_DSCR
        ,PROJ_STAT_CD
        ,FIRST_PAGE_URL AS CM_URL
        ,page_type AS PAGE_TYPE
        ,alert_id AS ALERT_ID
        ,VISITS
        ,ENG AS ENGAGED
       
        FROM CM_DRdf2

        WHERE ALERT_ID = 'DR_1'
        AND VISITS>9; """

CM_DRdf4 = (ps.sqldf(CM_DRdf3, locals()))
CM_DRdf4.head(5)

,PLAN_CD,PLAN_NAME,PLAN_DSCR,PROJ_CD,PROJ_NAME,PROJ_DSCR,PROJ_STAT_CD,CM_URL,PAGE_TYPE,ALERT_ID,VISITS,ENGAGED
0,PLN1164,Watson Customer Engagement,2019 Watson Customer Engagement,000029PM ...,000029PM_LA Watson Campaign Automation_Integrated,Integrated Campaign for LA lead generation,IN_PROGRESS,www.ibm.com/account/reg/es-es/signup?formid=ur...,None,DR_1,11541,328.0
1,PLN1123,Corporate Advertising,Corporate Advertising helps to drive engagemen...,000027JP ...,000027JP_CAdv_MasterBrand,Develop masterbrand content that will run on p...,IN_PROGRESS,researcher.watson.ibm.com/researcher/view_grou...,None,DR_1,8409,410.0
2,PLN1161,Systems,2019 Systems IBM Systems Marketing Operations...,000016HY ...,000016HY_GCG_Hybrid Cloud Storage_Storage Spec...,000016HY Hybrid Cloud Storage - CN,IN_PROGRESS,www.ibm.com/account/reg/cn-zh/signup?formid=ur...,None,DR_1,8375,259.0
3,PLN1153,Hybrid Cloud,2019 Hybrid Cloud,000022MY ...,000022MY JP DevOps and Hybrid IT Management,,IN_PROGRESS,www.ibm.com/jp-ja/cloud/info/webshere,None,DR_1,6813,326.0
4,PLN1160,Security,2019 Security,000030RQ ...,000030RQ LA Protect the mobile enterprise_MaaS360,,IN_PROGRESS,www.ibm.com/mx-es/campaign/maas360,None,DR_1,4637,701.0


In [34]:
len(CM_DRdf4)

877

In [35]:
CM_DRdf4.to_csv('C:\\Users\\IBM_ADMIN\\Documents\\Alerts\\ALERT 2 - Unregistered URLs\\unreg_20190202.csv', header=True, index=False, index_label=None)